In [28]:
# import libraries
import random
from collections import deque
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from sklearn import preprocessing
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Keras libraries
from tensorflow.keras.models import Sequential

In [29]:
PRECEDING_PRICES = 60  # preceeding 60 days prices
FUTURE_PRICES_PREDICT = 3  # how far into the future to predict
TO_PREDICT = "Bitcoin"

In [30]:
# read datasets and join them
data_directory: Path = Path("./cryptocurrency-historical-prices")
main_df = pd.DataFrame()
for file in data_directory.iterdir():
    if file.is_file() and file.suffix == ".csv":
        cryptocurrency_name = file.name.split("_")[1].split(".")[
            0
        ]  # Bitcoin, Aave, Cardano, Solana, Dogecoin, etc.
    df = pd.read_csv(f"{data_directory.name}/{file.name}", names = ["SNo", "Name", "Symbol", "Date", "High", "Low", "Open", "Close", "Volume", "Marketcap"], parse_dates=True)
    # rename to distinguish the cryptocurrency we are working with
    df.rename(columns={"Close": f"{cryptocurrency_name}_close", "Volume": f"{cryptocurrency_name}_volume"}, inplace=True)
    # set time as index so we can join the dataframes
    df.set_index("Date", inplace=True)
    # drop columns we are not interested in
    df = df[[f"{cryptocurrency_name}_close", f"{cryptocurrency_name}_volume"]]
    # df.fillna(method="ffill", inplace=True)
    main_df = df if len(main_df) == 0 else main_df.join(df)
# use previous valid value if there are gaps in the data
main_df.fillna(method="ffill", inplace=True) 
# main_df.fillna(method="bfill", inplace=True)
main_df.dropna(how="all", inplace=True)
print(main_df.head())

                     NEM_close    NEM_volume  EOS_close  EOS_volume  \
Date                                                                  
2015-04-02 23:59:59   0.000314  28549.400391        NaN         NaN   
2015-04-03 23:59:59   0.000310  20677.900391        NaN         NaN   
2015-04-04 23:59:59   0.000277  23550.199219        NaN         NaN   
2015-04-05 23:59:59   0.000232  26800.199219        NaN         NaN   
2015-04-06 23:59:59   0.000289  22511.500000        NaN         NaN   

                     Monero_close  Monero_volume  Polkadot_close  \
Date                                                               
2015-04-02 23:59:59      0.969260   88831.296875             NaN   
2015-04-03 23:59:59      0.952245  139293.000000             NaN   
2015-04-04 23:59:59      0.859290  128977.000000             NaN   
2015-04-05 23:59:59      0.883731  143439.000000             NaN   
2015-04-06 23:59:59      0.789463   38050.898438             NaN   

                     Polk

In [31]:
def make_order_decision(current_price, future_price):
	if float(current_price) < float(future_price): 
		return 1 # buy order
	else:
		return 0

In [32]:
# create column with the target future value
main_df["future_price_to_predict"] = main_df[f"{TO_PREDICT}_close"].shift(-FUTURE_PRICES_PREDICT) # negative to shift columnn up
# main_df["future_price_to_predict"] = main_df["future_price_to_predict"].astype(float)
main_df["order_decision"] = list(map(make_order_decision, main_df[f"{TO_PREDICT}_close"], main_df["future_price_to_predict"]))
main_df = main_df.dropna()
print(main_df.head())

                     NEM_close    NEM_volume  EOS_close    EOS_volume  \
Date                                                                    
2020-10-05 23:59:59   0.122335  3.187050e+07   2.529068  1.865500e+09   
2020-10-06 23:59:59   0.117298  2.983882e+07   2.693054  5.184280e+09   
2020-10-07 23:59:59   0.117939  2.089961e+07   2.646043  3.073509e+09   
2020-10-08 23:59:59   0.119316  2.296033e+07   2.609671  2.628172e+09   
2020-10-09 23:59:59   0.119971  2.442877e+07   2.645295  2.196491e+09   

                     Monero_close  Monero_volume  Polkadot_close  \
Date                                                               
2020-10-05 23:59:59    112.903248   7.640756e+08        4.172700   
2020-10-06 23:59:59    106.665996   7.967904e+08        3.756162   
2020-10-07 23:59:59    109.989031   9.658406e+08        3.858798   
2020-10-08 23:59:59    110.909933   8.347165e+08        4.102642   
2020-10-09 23:59:59    116.034673   8.691367e+08        4.262468   

           

In [33]:
# normalize data except for the order_decision column
def normalize_and_scale_df(df):
	# no longer need future_price_to_predict column
	df = df.drop("future_price_to_predict", axis=1)

	for column in df.columns:
		if column != "order_decision":
			# normalize data based on percentrage
			df[column] = df[column].pct_change()
			df.dropna(inplace=True)
			# scale values between 0 and 1
			df[column] = preprocessing.scale(df[column].values) 
	df.dropna(inplace=True)

	predictions_sequence = []
	previous_days_sequence = deque(maxlen=PRECEDING_PRICES)	

	for value in df.values:
		# do not include the target in the sequence
		previous_days_sequence.append([i for i in value[:-1]])
		# only keep last PRECEDING_PRICES observations
		if len(previous_days_sequence) == PRECEDING_PRICES:
			predictions_sequence.append([np.array(previous_days_sequence), value[-1]])
	# shuffle sequential data for good measure
	random.shuffle(predictions_sequence)

	buy_orders = []
	not_buy_orders = []

	for sequence, order_decision in predictions_sequence:
		if order_decision == 1:
			buy_orders.append([sequence, order_decision])
		else:
			not_buy_orders.append([sequence, order_decision])
	
	random.shuffle(buy_orders)
	random.shuffle(not_buy_orders)

	# ensure both buy and not buy orders are the same length
	shorter_sequence = min(len(buy_orders), len(not_buy_orders))
	buy_orders = buy_orders[:shorter_sequence]
	not_buy_orders = not_buy_orders[:shorter_sequence]

	# combine buy and not buy orders into predictions_sequence
	predictions_sequence = buy_orders + not_buy_orders
	random.shuffle(predictions_sequence)

	x_data = []
	y_data = []

	for sequence, order_decision in predictions_sequence:
		x_data.append(sequence) # sequence is the input
		y_data.append(order_decision) # buy or not to buy
	 
	return np.array(x_data), y_data

In [34]:
# data will not be shuffled due to the nature of the data, which is sequential
# taking sequences of data that do not come in the future is likely a bad idea
# make_order_decision will usually be the same for data points 1 minute apart
sorted_dates = sorted(main_df.index.values)
last_5_percent = sorted_dates[-int(len(sorted_dates) * 0.05)]

test_df = main_df[(main_df.index >= last_5_percent)]
print(f"Shape of test data: {test_df.shape}")
main_df = main_df[(main_df.index < last_5_percent)]
print(f"Shape of training data: {main_df.shape}")

# train test split
# normalize and scale training data
x_train, y_train = normalize_and_scale_df(main_df)
# normalize and scale test data
x_test, y_test = normalize_and_scale_df(test_df)

print(f"train data: {len(x_train)} test data: {len(x_test)}")
print(f"Train Dont buys: {y_train.count(0)}, buys: {y_train.count(1)}")
print(f"Test Dont buys: {y_test.count(0)}, buys: {y_test.count(1)}")

Shape of test data: (13, 48)
Shape of training data: (259, 48)


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required by the scale function.